In [4]:
import os

from google.colab import userdata

os.environ['GEMINI_API_KEY'] =  userdata.get('GEMINI_API_KEY')

In [5]:
!pip install -Uq crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.9 MB/s

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:


from crewai.flow.flow import Flow, start, listen


class MyFlow(Flow):

  @start()
  def function1(self):
    print("step1")

  @listen(function1)
  def function2(self):
    print("step2")

obj = MyFlow()
obj.kickoff()

 Flow started with ID: 79b6c79c-30bb-4385-b68a-5d342f111cb4
step1
step2


In [8]:

from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

llm1.call("What is the capital of France?")

'The capital of France is **Paris**.\n'

In [9]:
google_embedder = {
    "provider": "google",
    "config": {
         "model": "models/text-embedding-004",
         "api_key": userdata.get('GEMINI_API_KEY'),
         }
}

In [6]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = "Users name is Muhammad Qasim. He is 30 years old and lives in Karchi, Pakistan. He is working as Chief Data Scientist at CancerClarity LLC"
string_source = StringKnowledgeSource(
    content=content,
)


# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    memory=True,
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
    embedder=google_embedder

)

result = crew.kickoff(inputs={"question": "What city does Muhammad Qasim live in and how old is he?"})

# Agent: About User
## Task: Answer the following questions about the user: What city does Muhammad Qasim live in and how old is he?


# Agent: About User
## Final Answer: 
Muhammad Qasim lives in Karachi, Pakistan, and he is 30 years old.




In [7]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crew1'

In [9]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

In [8]:
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
# from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List, Optional

# Create An agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew =  Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    memory=True,
    # Long-term memory for persistence storage across sessions
    long_term_memory= LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage1.db"
        )
    ),

    # Short-term memory for current context using RAG
    short_term_memory= ShortTermMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/short_term1/"
        )
    ),

    # Entity memory for tracking key information about entites
    entity_memory=EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity1/"
        )
    ),

    verbose=True,
        )


# Integrating Mem0 for Enhanced User Memory

In [10]:
import os
from crewai import Crew, Process
from mem0 import MemoryClient

# Set environment variables for Mem0
os.environ["MEM0_API_KEY"] = userdata.get('MEM0_API_KEY')

# Step 1: Record preferences based on past conversation or user input
client = MemoryClient()

messages = [
    {"role": "user", "content": "Hi there! I'm planning a vacation and could use some advice."},
    {"role": "assistant", "content": "Hello! I'd be happy to help with your vacation planning. What kind of destination do you prefer?"},
    {"role": "user", "content": "I am more of a beach person than a mountain person."},
    {"role": "assistant", "content": "That's interesting. Do you like hotels or Airbnb?"},
    {"role": "user", "content": "I like Airbnb more."},
]
client.add(messages, user_id="john")

# Step 2: Create a Crew with User Memory

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)

task = Task(description="Answer the following questions about the user: {question}",
            expected_output="An answer to the question.",
            agent=agent)


crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    memory=True,
    verbose=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id":"john"}
    }
)

crew.kickoff(inputs={"question": "What is your favorite vacation destination?"})


/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


# Agent: About User
## Task: Answer the following questions about the user: What is your favorite vacation destination?


# Agent: About User
## Final Answer: 
I do not have personal preferences. However, based on your preferences, a beach house rental on a tropical island might be your favorite vacation destination.




/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


CrewOutput(raw='I do not have personal preferences. However, based on your preferences, a beach house rental on a tropical island might be your favorite vacation destination.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Answer the following questions about the user: What is your favorite vacation destination?', name=None, expected_output='An answer to the question.', summary='Answer the following questions about the user: What is your...', raw='I do not have personal preferences. However, based on your preferences, a beach house rental on a tropical island might be your favorite vacation destination.', pydantic=None, json_dict=None, agent='About User', output_format=<OutputFormat.RAW: 'raw'>)], token_usage=UsageMetrics(total_tokens=293, prompt_tokens=210, cached_prompt_tokens=0, completion_tokens=83, successful_requests=1))

# Tools

In [10]:
!pip install -Uq 'crewai[tools]'

In [11]:
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [12]:
embedder = {
    "provider":"google",
    "config": {
        "model": "modesls/text-embedding-004",
        "api_key": userdata.get('GEMINI_API_KEY')
    }

}

In [13]:
# from langchain_google_genai import ChatGoogleGenerativeAI

llm1 = LLM(model="gemini/gemini-2.0-flash")

In [ ]:
from re import search
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')


# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst  with a keen eye for market trends.',
    tools=[search_tool],
    llm=llm1,
    verbose=True
)


writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool,file_tool],
    verbose=True,
    llm=llm1
)


# Define tasks
research = Task(
    description='Research the lastest trends in the AI industry and provide a summary?',
    expected_output="A summary of the top 3 trending developments in the AI Industry with  a unique perspective on their significance.",
    agent=researcher

)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'   # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks = [research, write],
    verbose=True,
    planning=True, # Enable planning feature
    planning_llm=llm1
)

# Execute tasks
crew.kickoff()

 
[2025-03-05 06:47:06][INFO]: Planning the crew execution
 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfies the expected criteria, use the EXACT format below:

```
Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.

```
 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfies the expected criteria, use the EXACT format below:

```
Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.

```
# Agent: Market Research Analyst
## Tas

CrewOutput(raw='```markdown\n# The AI Revolution: Agentic, Multimodal, and Generative AI are Changing the Game\n\nThe artificial intelligence landscape is evolving at an unprecedented pace, with three key trends leading the charge: Agentic AI, Multimodal AI, and Generative AI. These advancements are not just incremental improvements; they represent fundamental shifts in how AI systems operate, perceive, and create. As these technologies mature, they promise to reshape industries, augment human capabilities, and unlock entirely new possibilities.\n\nAgentic AI, which focuses on creating autonomous systems capable of independent decision-making and problem-solving, is poised to revolutionize workflows. Imagine AI agents proactively managing tasks, adapting to changing circumstances, and collaborating seamlessly with humans. While the potential is immense, the real challenge lies in ensuring these agents align with human values, operate transparently, and maintain appropriate levels of co

In [ ]:
# prompt: import markdown for jupyter display

import IPython
import markdown

# Assuming the content of your markdown file is in the variable 'markdown_content'
# For example, if the file is named 'new_post.md' in your 'blog-posts' directory:
with open('./blog-posts/new_post.md', 'r') as f:
  markdown_content = f.read()


# Convert the markdown content to HTML
html = markdown.markdown(markdown_content)

# Display the HTML in the Jupyter Notebook
IPython.display.HTML(html)


## Creating your own Tools

In [40]:
# #First Method of creating tool
# from crewai.tools import BaseTool
# from pydantic import BaseModel, Field
# from typing import Type

# class MyToolInput(BaseModel):
#     """Input Schema for MyCustomTool."""
#     stuent_name: str = Field(...,description="Student name")
#     student_roll_no: int = Field(...,description="student id")


# class PiaicStudentCard(BaseTool):
#     name: str = "Piaic student card generator"
#     description: str = "this function will create Piaic student card"
#     args_schema: Type[BaseModel] = MyToolInput

#     def _run(self, student_name: str, student_roll_no: int) ->str:
#       # Your tools logic here
#       return f"""PIAIC Student Card
#       Student name: {student_name}
#       student roll no: {student_roll_no}
#       Pakistan Zindabad!
#       """


from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

class MyToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    student_name: str = Field(..., description="Student name")
    student_roll_no : int = Field(..., description="student id")

class PiaicStudentCard(BaseTool):
    name: str = "Piaic student card generator"
    description: str = "this function will create Piaic student card"
    args_schema: Type[BaseModel] = MyToolInput

    def _run(self, student_name: str, student_roll_no: int ) -> str:
        # Your tool's logic here
        return f"""PIAIC student card
student name: {student_name}
student roll no: {student_roll_no}
Pakistan zindabd!
        """

In [41]:
#2nd Method of  creating tool
from crewai.tools import tool

@tool("PIAIC fee update")
def my_tool(roll_no: int) ->dict | str:
  """this function search piaic student fee updates it will required roll of PIAIC student"""
  #database


  data = {100: "paid",
          200: "unpaid"}

  status = data.get(roll_no)
  if status:
    return {"status": status}
  else:
    return "student not found"

In [42]:
from crewai import Agent, Task, Crew

card = PiaicStudentCard()


piaic_manager = Agent(
    role="PIAIC manager",
    goal = "Manage all quries regarding PIAIC and you will use only relevant tools for student query",
    backstory="""You are a master at understanding people and their preferences.""",
    tools=[card, my_tool],
    verbose=True,
    llm=llm1
)

piaic_card_creator = Task(
    description="you will be responsible for all PIAIC relevant operations, student query '{query}' you must be know how to answer his question based on final context",
    expected_output="final query answer only",
    agent=piaic_manager
)

crew = Crew(
    agents=[piaic_manager],
    tasks=[piaic_card_creator],
    verbose=True
)

result = crew.kickoff(inputs={
    "query":"I'm PIAIC student my name is Muhammad Qasim and my roll number is 100, can you create my student card."
})

print(result)


# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student my name is Muhammad Qasim and my roll number is 100, can you create my student card.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Thought: The user is asking to generate a student card. I have the student's name and roll number, so I can use the "Piaic student card generator" tool.
## Using tool: Piaic student card generator
## Tool Input: 
"{\"student_name\": \"Muhammad Qasim\", \"student_roll_no\": 100}"
## Tool Output: 
PIAIC student card
student name: Muhammad Qasim
student roll no: 100
Pakistan zindabd!
        


# Agent: PIAIC manager
## Final Answer: 
PIAIC student card
student name: Muhammad Qasim
student roll no: 100
Pakistan zindabd!


PIAIC student card
student name: Muhammad Qasim
student roll no: 100
Pakistan zindabd!


In [44]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 200, can you share my fee updates?."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 200, can you share my fee updates?.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Using tool: PIAIC fee update
## Tool Input: 
"{\"roll_no\": 200}"
## Tool Output: 
{'status': 'unpaid'}


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Piaic student card generator
Tool Arguments: {'student_name': {'description': 'Student name', 'type': 'str'}, 'student_roll_no': {'description': 'student id', 'type': 'int'}}
Tool Description: this function will create Piaic student card
Tool Name: PIAIC fee update
Tool Arguments: {'roll_no': {'description': None, 'type': 'int'}}
Tool Description: this function search piaic student fee updates it will required roll of PIAIC student

IMPORTANT: Use the following format in your response:

```
Thought: 

# Process

In [14]:
import os
from crewai import Agent, Task, Crew, Process

# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)



# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts2')
file_tool = FileReadTool()
search_tool = SerperDevTool()
# web_rag_tool = WebsiteSearchTool()

#Define your agents
researcher = Agent(
    role="Researcher",
    goal="Conduct thorough research and analysis on AI and AI Agents",
    backstory="You're an expert researcher, specialized in technology, software engineering, AI and startups. You work as a freelancer and are currently researching for a new client.",
    allow_delegation=False,
    tools=[search_tool],
    llm=llm1
)

writer = Agent(
    role="Senior Writer",
    goal="Create compelling content about AI and AI agents",
    backstory="You're a senior writer, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently writing content for a new client.",
    allow_delegation=False,
    tools=[docs_tool, file_tool],
    llm=llm1
)


# Define your task
task = Task(
    description="Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with thier paragraphs and your notes.",
    expected_output="5 bullet points, each with a paragraph and accompanying notes.",
)

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    llm=llm1
)

#Intantiate your crew with a custom manager
crew = Crew(
    agents=[researcher,writer],
    tasks=[task],
    manager_agent=manager,
    function_calling_llm=llm1,
    process=Process.hierarchical,
    verbose=True,
)

#Start the crew's work
result=crew.kickoff()
from IPython.display import display, Markdown
display(Markdown(result))

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

# Agent: Project Manager
## Task: Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with thier paragraphs and your notes.


# Agent: Project Manager
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": {\"description\": \"Research and provide a list of 5 trending topics or interesting areas that could be developed into engaging articles. For each suggested topic, provide a brief explanation of why it's currently relevant or interesting. Focus on topics that can capture a wide audience and are suitable for a general interest blog.\"}, \"context\": {\"description\": \"I need you to identify potential topics for articles. I will take this research and create the articles, you just need to provide the topics and why they are relevant.\"}, \"coworker\": \"Researcher\"}"
## Tool Output: 

I encountered an error while trying to use the 

TypeError: Markdown expects text, not CrewOutput(raw="*   **AI Agents: The rise of AI agents that can independently perform tasks.**\n\nImagine a world where AI isn't just a tool, but an autonomous partner capable of handling complex tasks, learning from experience, and making independent decisions. AI agents are rapidly evolving from science fiction to reality, promising to revolutionize industries from customer service to logistics. But as these digital entities become more sophisticated, questions arise about control, responsibility, and the very nature of work itself.\n\n*   **Notes:** General tech enthusiasts, business leaders, those curious about the future of work. Explore use cases in different industries, the potential for job displacement, and the challenges of governing autonomous AI.\n\n*   **Generative AI and its Democratization: Generative AI is rapidly becoming more accessible to the general public.**\n\nFrom crafting stunning visuals to composing catchy tunes and even w

In [15]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

*   **AI Agents: The rise of AI agents that can independently perform tasks.**

Imagine a world where AI isn't just a tool, but an autonomous partner capable of handling complex tasks, learning from experience, and making independent decisions. AI agents are rapidly evolving from science fiction to reality, promising to revolutionize industries from customer service to logistics. But as these digital entities become more sophisticated, questions arise about control, responsibility, and the very nature of work itself.

*   **Notes:** General tech enthusiasts, business leaders, those curious about the future of work. Explore use cases in different industries, the potential for job displacement, and the challenges of governing autonomous AI.

*   **Generative AI and its Democratization: Generative AI is rapidly becoming more accessible to the general public.**

From crafting stunning visuals to composing catchy tunes and even writing compelling articles, generative AI is putting creative power into the hands of anyone with an internet connection. Once confined to research labs and tech giants, these powerful tools are now democratizing content creation, sparking a wave of innovation and raising fundamental questions about authorship, originality, and the future of art.

*   **Notes:** Creatives, artists, marketers, educators, and anyone interested in exploring new forms of expression. Explore the impact on traditional creative industries, the potential for misinformation, and the ethical considerations of AI-generated content.

*   **AI and Cybersecurity: With the increasing reliance on AI, cybersecurity threats are also evolving.**

As we increasingly rely on AI to power our digital lives, a new breed of cyber threats is emerging – threats that leverage AI itself. From AI-powered phishing attacks that can mimic human behavior to autonomous malware that can evade traditional defenses, the cybersecurity landscape is rapidly evolving. Staying ahead of these AI-driven threats requires a new approach to security, one that embraces AI as both a shield and a sword.

*   **Notes:** IT professionals, cybersecurity experts, business leaders concerned about data security, and anyone interested in the future of cyber warfare. Explore the different types of AI-powered threats, the challenges of defending against them, and the role of AI in cybersecurity solutions.

*   **The Ethical Implications of AI: AI raises a host of ethical concerns.**

Artificial intelligence is no longer a futuristic fantasy; it's a present-day reality woven into the fabric of our lives. But with this rapid advancement comes a complex web of ethical dilemmas. From biased algorithms perpetuating societal inequalities to autonomous weapons systems making life-or-death decisions, AI raises profound questions about fairness, accountability, and the very future of humanity.

*   **Notes:** General public, policymakers, ethicists, academics, and anyone concerned about the societal impact of technology. Explore specific ethical dilemmas such as algorithmic bias, autonomous weapons, and the impact of AI on privacy and autonomy.

*   **AI in Healthcare: AI is transforming healthcare in numerous ways, from diagnosing diseases to personalizing treatment plans.**

Imagine a world where diseases are detected earlier, treatments are tailored to individual needs, and healthcare is more accessible and affordable for all. AI is revolutionizing healthcare, enabling doctors to diagnose illnesses with greater accuracy, personalize treatment plans based on individual genetic profiles, and develop new drugs and therapies at unprecedented speeds. But with these advancements come questions about data privacy, algorithmic bias, and the role of human empathy in patient care.

*   **Notes:** Healthcare professionals, patients, researchers, policymakers, and anyone interested in the future of medicine. Explore specific applications of AI in healthcare, such as diagnosis, drug discovery, and personalized medicine, as well as the ethical and societal implications of AI in healthcare.
```

# Advance Tools

In [17]:
from pydantic import BaseModel, Field

class TeacherAssignmentOutPut(BaseModel):
  score: int = Field(..., description="student assignment score")
  feedback: str = Field(..., description="teacher feedback")
  question: str = Field(..., description="question")
  answer: str = Field(..., description="python code for markdown format")

In [21]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import CodeInterpreterTool


teacher = Agent(role="AI teacher",
                goal="you are Agentic AI teacher you have to check python assignment code submission.",
                backstory="you have to check student python code submission.",
                tools=[CodeInterpreterTool()],
                llm=llm1)

assignment_check = Task(
    description="you are Agentic AI teacher you have to check python assignment code submission. question: '''{question}''' student solution : '''{solution}'''",
    expected_output="final code is runing assign number between 1-10",
    output_pydantic=TeacherAssignmentOutPut,
    agent=teacher # Added the missing agent here
)

crew = Crew(
    agents=[teacher],
    tasks=[assignment_check],
    process=Process.sequential,
    verbose=True
)

question = "create function add two numbers"

solution = """
def add_two_numbers(num1: int, num2: int) -> int:
  '''Adds two numbers together.

  Args:
    num1: The first number.
    num2: The second number.

  Returns:
    The sum of the two numbers.
  '''
  return num1 + num2
"""

result = crew.kickoff(inputs={"question": question, "solution": solution})

print(result)



# Agent: AI teacher
## Task: you are Agentic AI teacher you have to check python assignment code submission. question: '''create function add two numbers''' student solution : '''
def add_two_numbers(num1: int, num2: int) -> int:
  '''Adds two numbers together.

  Args:
    num1: The first number.
    num2: The second number.

  Returns:
    The sum of the two numbers.
  '''
  return num1 + num2
'''


# Agent: AI teacher
## Using tool: Code Interpreter
## Tool Input: 
"{\"code\": \"def add_two_numbers(num1: int, num2: int) -> int:\\n  '''Adds two numbers together.\\n\\n  Args:\\n    num1: The first number.\\n    num2: The second number.\\n\\n  Returns:\\n    The sum of the two numbers.\\n  '''\\n  return num1 + num2\\n\\n# Test the function\\nresult1 = add_two_numbers(5, 3)\\nprint(f'5 + 3 = {result1}')\\n\\nresult2 = add_two_numbers(1, 10)\\nprint(f'1 + 10 = {result2}')\", \"libraries_used\": []}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the erro

In [ ]:
# prompt: write a function of adding two numbers with pydocs

def add_two_numbers(num1: int, num2: int) -> int:
  """Adds two numbers together.

  Args:
    num1: The first number.
    num2: The second number.

  Returns:
    The sum of the two numbers.
  """
  return num1 + num2


In [22]:
result.to_dict()

{'score': 5,
 'feedback': 'The code is correct and well-documented. The function `add_two_numbers` correctly adds two integers and includes a proper docstring. The type hints are also correctly implemented. The code fulfills all requirements.',
 'question': 'create function add two numbers',
 'answer': "def add_two_numbers(num1: int, num2: int) -> int:\n  '''Adds two numbers together.\n\n  Args:\n    num1: The first number.\n    num2: The second number.\n\n  Returns:\n    The sum of the two numbers.\n  '''\n  return num1 + num2"}

In [23]:
for k,v in result.to_dict().items():
  print(str("*"*10) + str(k) + str("*"*10))
  display(v)

**********score**********


5

**********feedback**********


'The code is correct and well-documented. The function `add_two_numbers` correctly adds two integers and includes a proper docstring. The type hints are also correctly implemented. The code fulfills all requirements.'

**********question**********


'create function add two numbers'

**********answer**********


"def add_two_numbers(num1: int, num2: int) -> int:\n  '''Adds two numbers together.\n\n  Args:\n    num1: The first number.\n    num2: The second number.\n\n  Returns:\n    The sum of the two numbers.\n  '''\n  return num1 + num2"